# MDF Final Project - Fetching Presidential Approval Ratings

## Setup

In [1]:
import pyspark
import pandas as pd
import numpy as np
import bs4
#pip install html5lib
#pip install lxml

## Getting tables for presidential approval ratings

Presidential approval rating data comes from the American Presidency Project, a non-profit and non-partisan institution with a focus on the legacies of US presidents. More info here: https://www.presidency.ucsb.edu/statistics/data/presidential-job-approval-all-data

In [3]:
# Defining Function for web scraping and table extraction
def webscrape_table(url):
    '''Get tables from URL'''
    tables = pd.read_html(url)
    '''If the webpage has multiple tables, choose the desired one'''
    table = tables[0]
    '''Return the extracted table'''
    return(table)


In [4]:
# Bush (2)
bush = webscrape_table("https://www.presidency.ucsb.edu/statistics/data/george-w-bush-public-approval")

# Obama
obama = webscrape_table("https://www.presidency.ucsb.edu/statistics/data/barack-obama-public-approval")

# Trump (1st term) table
trump1 = webscrape_table("https://www.presidency.ucsb.edu/statistics/data/donald-j-trump-public-approval")

# Biden table
biden = webscrape_table("https://www.presidency.ucsb.edu/statistics/data/joseph-r-biden-public-approval")

# Trump (2nd term) table
trump2 = webscrape_table("https://www.presidency.ucsb.edu/statistics/data/donald-j-trump-2nd-term-public-approval")


## Cleaning tables

In [5]:
# Adding party labels
bush['party'] = "R"
trump1['party'] = "R"
trump2['party'] = "R"
obama['party'] = "D"
biden['party'] = "D"

In [6]:
# Preliminary cleaning
biden['Start Date'] = biden['Start Date'].replace("12/1/2/021", "12/1/2021")
trump1 = trump1.dropna(subset=["Start Date"])
obama = obama.dropna(subset=["Start Date"])

In [7]:
# Defining functions to reformat tables (pivoting dates to long and adding in dates)

In [8]:
def reformat_table(df):
    '''Formatting date columns to date-time'''
    df["Start Date"] = pd.to_datetime(df["Start Date"], format='mixed', errors="coerce")
    df["End Date"] = pd.to_datetime(df["End Date"], format='mixed', errors="coerce")

    '''Expand date ranges into individual dates'''
    expanded_df = df.apply(lambda row: pd.DataFrame({
        'date': pd.date_range(row['Start Date'], row['End Date']),
        'Approving': row['Approving'],
        'Disapproving': row['Disapproving'],
        'Unsure/NoData': row['Unsure/NoData'],
        'party' : row['party']
    }), axis=1)

    '''Convert list of dataframes to a single dataframe'''
    expanded_df = pd.concat(expanded_df.tolist(), ignore_index=True)

    '''Return new df'''
    return(expanded_df)

In [9]:
# Reformating
bush_clean = reformat_table(bush)
obama_clean = reformat_table(obama)
trump1_clean = reformat_table(trump1)
biden_clean = reformat_table(biden)
trump2_clean = reformat_table(trump2)

In [10]:
# Binding
approval = pd.concat([bush_clean, 
                      obama_clean,
                      trump1_clean, 
                      biden_clean,
                      trump2_clean], axis=0)

In [11]:
# Exporting
approval.to_csv('approval.csv', index=False)

In [12]:
approval.head()

,date,Approving,Disapproving,Unsure/NoData,party
0,2009-01-09,34.0,61.0,5.0,R
1,2009-01-10,34.0,61.0,5.0,R
2,2009-01-11,34.0,61.0,5.0,R
3,2008-12-12,29.0,67.0,4.0,R
4,2008-12-13,29.0,67.0,4.0,R
